In [189]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind as ttest
import seaborn

In [197]:
# Read in CSV

df = pd.read_csv('final_dataset.csv')

In [198]:
# Remove bad entries

original_length = len(df.index)
dropped = 0
for i in range(original_length):
    index = original_length - (i + 1)
    if np.isnan(df["apple_stars"][index]):
        df = df.drop(index)
        dropped += 1
    else:
        for column in df.columns:
            if df[column][index] == "[]":
                df = df.drop(index)
                dropped += 1
                break
print("Dropped %d" % dropped)

Dropped 24


In [199]:
# Re-index df

index_col = "Unnamed: 0"
try:
    del df[index_col]
    df = df.reset_index()
    del df["index"]
    df.index.name = None
except KeyError:
    print("Couldn't find pre-existing index column.")

In [211]:
# Verifying our t tests from RStudio

# This library doesn't use the exact same t-test (and the data is cleaned slightly differently),
# but the p-values remain relatively similar. Those which we claimed are significant remain
# significant (most importantly).

attributes = ["F0semitoneFrom27.5Hz_sma3nz_amean", "jitterLocal_sma3nz_amean",
              "shimmerLocaldB_sma3nz_amean", "HNRdBACF_sma3nz_amean", "loudness_sma3_amean"]

results_df = pd.DataFrame(columns=['attribute', 'stars_t', 'stars_p', 'ratings_t', 'ratings_p'])

for attribute in attributes:

    test_df = pd.DataFrame(columns=[attribute, 'stars', 'ratings'])

    for i in range(len(df[attribute])):
        stars = float(df["apple_stars"][i])
        ratings = float(df["apple_ratings"][i])
        attribute_value = float(df[attribute][i])
        if not np.isnan(stars):
            test_df.loc[len(test_df.index)] = [attribute_value, stars, ratings]

    # attribute_median = test_df.median()[0]
    stars_low_limit, stars_high_limit = test_df["stars"].quantile([0.5, 0.5])
    ratings_low_limit, ratings_high_limit = test_df["ratings"].quantile([0.5, 0.5])

    for i in range(len(test_df[attribute])):

        if test_df["stars"][i] >= stars_high_limit:
            test_df.at[i, "stars"] = "H"
        elif test_df["stars"][i] <= stars_low_limit:
            test_df.at[i, "stars"] = "L"
        else:
            test_df.at[i, "stars"] = "M"

        if test_df["ratings"][i] >= ratings_high_limit:
            test_df.at[i, "ratings"] = "H"
        elif test_df["ratings"][i] <= ratings_low_limit:
            test_df.at[i, "ratings"] = "L"
        else:
            test_df.at[i, "ratings"] = "M"

    stars_g1 = test_df[test_df["stars"] == "H"]
    stars_g2 = test_df[test_df["stars"] == "L"]

    t_stars, p_stars = ttest(stars_g1[attribute], stars_g2[attribute], alternative='two-sided', nan_policy='omit')

    ratings_g1 = test_df[test_df["ratings"] == "H"]
    ratings_g2 = test_df[test_df["ratings"] == "L"]

    t_ratings, p_ratings = ttest(ratings_g1[attribute], ratings_g2[attribute], alternative='two-sided', nan_policy='omit')

    results_df.loc[len(results_df.index)] = [attribute, t_stars, p_stars, t_ratings, p_ratings]
